In [2]:
import numpy as np
import pandas as pd
from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from datasets import Dataset
import pickle
from utils.helper import read_py150k_code, read_file_to_string
import regex as re

fname_prefix = "/data/users/team2_capstone/code-style-probing/"

In [6]:
combined_df = pd.read_csv(fname_prefix + "/data/combined_data_uncommented.csv")
combined_df = combined_df[combined_df['uncommented_content'].notnull()]
dataset = Dataset.from_pandas(combined_df).train_test_split(test_size=0.2)

/tmp/ipykernel_11016/4092686552.py:1: DtypeWarning: Columns (84,85,86,87) have mixed types. Specify dtype option on import or set low_memory=False.
  combined_df = pd.read_csv(fname_prefix + "/data/combined_data_uncommented.csv")


In [7]:

display (combined_df)

,Unnamed: 0.1,Unnamed: 0,line_count,comment_count,comment_total_len,comment_avg_len,comment_density,id_total,lower_case,id_total_var,...,filepath,forks,issue_events,stars,parse_error,func_async_count,file,labels,content,uncommented_content
0,0,0,116.0,2.0,44.0,22.000000,0.017241,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/00/wikihouse/urls.py,19,#!/usr/bin/env python\n# -*- coding: utf-8 -*-...,""""""" Provides ``mapping`` of url paths to reque..."
1,1,1,363.0,24.0,1244.0,51.833333,0.066116,381.0,214.0,333.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/0rpc/zerorpc-python/zerorpc/events.py,-1,# -*- coding: utf-8 -*-\n# Open Source Initiat...,import msgpack\nimport gevent.pool\nimport gev...
2,2,2,13.0,1.0,21.0,21.000000,0.076923,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/0xadada/dockdj/app/manage.py,19,"#!/usr/bin/env python\n""""""Django's command lin...","""""""Django's command line utility.""""""\nimport o..."
3,3,3,34.0,0.0,0.0,0.000000,0.000000,26.0,18.0,26.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/1stvamp/hippybot/setup.py,16,"""""""Installer for hippybot\n""""""\n\nimport os\nc...","""""""Installer for hippybot\n""""""\nimport os\ncwd..."
4,4,4,11.0,1.0,21.0,21.000000,0.090909,3.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,data/2buntu/2buntu-blog/manage.py,19,#!/usr/bin/env python\nimport os\nimport sys\n...,import os\nimport sys\nif __name__ == '__main_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215045,215045,115045,1292.0,201.0,7829.0,38.950249,0.155573,1658.0,1533.0,1646.0,...,latools/helpers/plot.py,11.0,77.0,9.0,1.0,NaN,oscarbranson/latoolslatools/helpers/plot.py,-1,"""""""\nPlotting functions.\n\n(c) Oscar Branson ...","""""""\nPlotting functions.\n\n(c) Oscar Branson ..."
215046,215046,115046,194.0,17.0,633.0,37.235294,0.087629,63.0,42.0,57.0,...,library/f5bigip_ltm_monitor_snmp_dca.py,5.0,72.0,6.0,1.0,NaN,erjac77/ansible-module-f5bigiplibrary/f5bigip_...,-1,#!/usr/bin/python\n# -*- coding: utf-8 -*-\n#\...,"ANSIBLE_METADATA = {'metadata_version': '1.1',..."
215047,215047,115047,145.0,1.0,23.0,23.000000,0.006897,111.0,76.0,99.0,...,test/test_config.py,139.0,376.0,754.0,1.0,NaN,python-hyper/hyper-h2test/test_config.py,-1,"# -*- coding: utf-8 -*-\n""""""\ntest_config\n~~~...","""""""\ntest_config\n~~~~~~~~~~~\n\nTest the conf..."
215048,215048,115048,48.0,0.0,NaN,0.000000,0.000000,53.0,44.0,47.0,...,tests/trade/test_trade.py,14.0,198.0,42.0,1.0,NaN,techbureau/zaifbottests/trade/test_trade.py,-1,import unittest\nfrom zaifbot.trade.trade impo...,import unittest\nfrom zaifbot.trade.trade impo...


In [13]:
from transformers import RobertaTokenizer, T5ForConditionalGeneration, PLBartTokenizer

tokenizer = RobertaTokenizer.from_pretrained('Salesforce/codet5-base')
example =dataset['train']['uncommented_content']
#label = tokenizer(example, truncation=True)
#print (len(label['input_ids']))
def tokenization(example):
    return_dict = tokenizer(example['uncommented_content'], padding='max_length', truncation=True)
    #print ((example['content'][10]))
    labels = tokenizer(example['content'], padding='max_length', truncation=True).input_ids
    return_dict['labels'] = labels
    #print (return_dict.keys())
    
    return return_dict
train_dataset = dataset["train"].map(tokenization, batched=True)
train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
train_dataset.format['type']

test_dataset = dataset["test"].map(tokenization, batched=True)
test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_dataset.format['type']

loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/vocab.json from cache at /soe/ksmunson/.cache/huggingface/transformers/1e2aacf615bc83f25a9d748eccb762b335eee01a29ab7a8db9b8e86cc851d489.9a48c5abf25554713c6513ab01066e53569b9a2da0d6189715951cf7c6288805
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/merges.txt from cache at /soe/ksmunson/.cache/huggingface/transformers/7eaa9b856402f05e8fdd452951872ecd3c2692ea9abb86b7ab62b07e3bc5f7de.7179059568f1a130b0a79e4bac71f38545207cab0ec45ce82ca09afadb2649a3
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/added_tokens.json from cache at /soe/ksmunson/.cache/huggingface/transformers/a3e93db547e41cdd21f01826d07c5679e111b02d8e969c607611c30a6acbe191.5cc6e825eb228a7a5cfd27cb4d7151e97a79fb962b31aaf1813aa102e746584b
loading file https://huggingface.co/Salesforce/codet5-base/resolve/main/special_tokens_map.json from cache at /soe/ksmunson/.cache/huggingface/transformers/5941df5e4315c5ab63

  0%|          | 0/162 [00:00<?, ?ba/s]

  0%|          | 0/41 [00:00<?, ?ba/s]

'torch'

In [15]:
print ((train_dataset[0]))
train_dataset.save_to_disk(fname_prefix + "datasets/codet5_train_uncommented_combined_padded.hf")
test_dataset.save_to_disk(fname_prefix + "datasets/codet5_test_uncommented_combined_padded.hf")
#print (combined_df[combined_df['content'] ==  "File Error"])
#combined_df.to_csv(fname_prefix + 'data/labeled_code/full_data.csv')

{'labels': tensor([    1,  2080,  3207,   715,    18,  1969,  3404, 13373,  1930,  3360,
         3448, 12074,   559,   487,   389,  2171,  3448, 12074,   559,   203,
         5666,  1610,   487,   389,  3530,   203,   203,  1106,  3867,  5776,
        24899,  2171,  3448, 12074,   559,  4672,   203,   565,   389,  2938,
           67,   803,    67,   701,   273,   296, 31320,  3915,  3175,    80,
           18,  8109,    11,   203,   565,   389,   803,    67,   701,   273,
          296, 31320,  3915,  3175,    80,    18,  8109,    18,   955,  5776,
           11,   203,   565,   389,   877,    67,  9693,   273, 13666,  3266,
           11,    97,   203,   203,   565,   632,  4468,   203,   565,  1652,
         2036,    12,  2890,  4672,   203,  3639,  3536,   203,  3639, 11511,
          326,   977,  3512,  2036,   434,  3170,  3143,    18,   203,   203,
         3639,  1021,   296,  3266,    11,  1272,   353,   279,  2036,   471,
         2026,   506,  1269,   487,    30,   203,  18